In [2]:
import torch
import numpy as np
import open3d as o3d
import open3d.ml.torch as ml3d
import open3d.ml as o3dml

In [3]:
print(torch.__version__)
print(torch.cuda.is_available())
print(np.__version__)
print(o3d.__version__)

2.2.2+cu121
True
1.26.4
0.19.0


In [ ]:
import numpy as np
import yaml
from pathlib import Path


# poss_dataset.py
import numpy as np
import yaml
import open3d.ml.torch as ml3d

# POSS (17)
POSS_LABELS = {
    0: "unlabeled",
    4: "1 person",
    5: "2+ person",
    6: "rider",
    7: "car",
    8: "trunk",
    9: "plants",
    10: "traffic sign 1", # standing sign
    11: "traffic sign 2", # hanging sign
    12: "traffic sign 3", # high/big hanging sign
    13: "pole",
    14: "trashcan",
    15: "building",
    16: "cone/stone",
    17: "fence",
    21: "bike",
    22: "ground"} # class definition

class POSSDataset(ml3d.datasets.SemanticKITTI):

    def __init__(
        self,
        dataset_path: str,
        poss_yaml_path: str, # <--- poss.yaml
        name: str = "poss",
        cache_dir: str = "./logs/cache_poss",
        use_cache: bool = False,
        class_weights=None,
        ignored_label_inds=None,
        test_result_folder=None,
        test_split=None,
        training_split=None,
        validation_split=None,
        all_split=None,
        **kwargs,
    ):
        super().__init__(dataset_path=dataset_path,
                         name=name,
                         cache_dir=cache_dir,
                         use_cache=use_cache,
                         class_weights=class_weights,
                         ignored_label_inds=ignored_label_inds or [],
                         test_result_folder=test_result_folder,
                         training_split=training_split or ["00","01","02","03"],
                         validation_split=validation_split or ["04"],
                         test_split=test_split or ["05"],
                         all_split=all_split or ["00","01","02","03","04","05"],
                         **kwargs)

        self._poss_yaml_path = poss_yaml_path
        with open(self._poss_yaml_path, "r") as f:
            DATA = yaml.safe_load(f)

        self.label_to_names = self.get_label_to_names()
        self.num_classes = len(self.label_to_names)

        # learning_map_inv: train_id -> raw_id
        remap_dict_inv = DATA["learning_map_inv"]
        max_key = max(remap_dict_inv.keys()) if remap_dict_inv else 0
        remap_lut = np.zeros((max_key + 100), dtype=np.int32)
        remap_lut[list(remap_dict_inv.keys())] = list(remap_dict_inv.values())

        # learning_map: raw_id -> train_id
        remap_dict = DATA["learning_map"]
        max_key_val = max(remap_dict.keys()) if remap_dict else 0
        remap_lut_val = np.zeros((max_key_val + 100), dtype=np.int32)
        remap_lut_val[list(remap_dict.keys())] = list(remap_dict.values())

        self.remap_lut = remap_lut
        self.remap_lut_val = remap_lut_val

    @staticmethod
    def get_label_to_names():
        return dict(POSS_LABELS)


In [5]:
ds = POSSDataset(
    dataset_path="./SemanticPOSS_dataset",
    poss_yaml_path="randlanet_poss.yml"
)

In [6]:
split = ds.get_split("train")
sample = split.get_data(0)
points = sample["point"].astype(np.float32)
label = sample["label"].astype(np.int32)

print(f"Points shape: {points.shape}")
print(f"Labels shape: {label.shape}")

Points shape: (66658, 3)
Labels shape: (66658,)


In [7]:
names = ds.label_to_names
num_classes = int(max(names.keys())) + 1
rng = np.random.default_rng(0)
palette = rng.random((num_classes, 3))

In [25]:
points_xyz = sample["point"].astype(np.float32)   # (N, 4) - координаты + интенсивность

In [26]:
data_infer = {
    "point": points_xyz,      # Основной тензор точек для модели
    "full_point": points_xyz  # ОБЯЗАТЕЛЬНОЕ ПОЛЕ для SemanticSegmentation
}

In [27]:
points_xyz = sample["point"].astype(np.float32)
print(f"Размерность points_xyz: {points_xyz.shape}")  # Должно быть (N, 4)
print(f"Количество признаков: {points_xyz.shape[1]}")

Размерность points_xyz: (66534, 3)
Количество признаков: 3


In [28]:
# Проверьте размерность в тестовых данных
test_split = ds.get_split("test")
sample = test_split.get_data(0)  # первый файл из папки 04
points = sample["point"].astype(np.float32)
print(f"Размерность тестовых данных (папка 04): {points.shape}")

Размерность тестовых данных (папка 04): (66534, 3)


In [15]:
print(f"in_channels: {cfg.model.get('in_channels')}")
print(f"num_classes: {cfg.model.get('num_classes')}")
print(f"ignored_label_inds: {cfg.model.get('ignored_label_inds')}")
print(f"dataset_path: {cfg.get('dataset_path')}")
print(f"test_split: {cfg.get('test_split')}")
print(f"training_split: {cfg.get('training_split')}")

in_channels: 4
num_classes: 15
ignored_label_inds: [0]
dataset_path: None
test_split: None
training_split: None


### Из за того что у меня стоит видеокарта нового поколения 5070 есть проблемы с совместью библиотек. Ниже начал обучение на cpu. По причине долгого обучения на cpu с вашего позволения не буду обучать модель до конца. 

In [ ]:
res = pipeline.run_inference(data_infer)

/media/daniil/d73a71ee-06a1-4243-882c-225fb09917a1/daniil/Learn/Semestr_3/3D/HomeWork/Homework_11/.venv/lib/python3.11/site-packages/torch/cuda/__init__.py:218: UserWarning: 
NVIDIA GeForce RTX 5070 Ti with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_50 sm_60 sm_70 sm_75 sm_80 sm_86 sm_90.
If you want to use the NVIDIA GeForce RTX 5070 Ti GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(


RuntimeError: Wrong feature dimension, please update in_channels(3 + feature_dimension) in config

In [55]:
# # 1. Загружаем конфигурацию
cfg_file = 'randlanet_poss.yml'
cfg = o3dml.utils.Config.load_from_file(cfg_file)

In [56]:
# 2. Получаем классы по именам из конфига
DatasetClass = POSSDataset  # Используем наш кастомный класс
ModelClass = o3dml.utils.get_module("model", cfg.model.name, "torch")
PipelineClass = o3dml.utils.get_module("pipeline", cfg.pipeline.name, "torch")

In [57]:
# 3. Создаем экземпляр датасета, используя параметры из конфига
print("Загружаем датасет POSS...")
# Извлекаем путь к датасету и другие параметры
dataset_path = cfg.dataset.pop('dataset_path', None)
dataset = DatasetClass(dataset_path, 
                        "randlanet_poss.yml",
                        **cfg.dataset)  # POSSDataset сам прочитает YAML

Загружаем датасет POSS...


In [59]:
# 4. Создаем модель и пайплайн
print("Создаем модель и pipeline...")
model = ModelClass(**cfg.model)

# Убедимся, что параметр 'dataset' (как строка) не передается в пайплайн
pipeline_cfg = cfg.pipeline.copy()
# Если он там есть, удаляем, т.к. передаем объект датасета явно
pipeline_cfg.pop('dataset', None)

pipeline = PipelineClass(
    model=model,
    dataset=dataset,  # Передаем объект датасета
    device='cpu',    # или 'cpu'
    **pipeline_cfg
)

Создаем модель и pipeline...


In [60]:
# 5. Обучение модели
print("Начинаем обучение...")
pipeline.run_train()

Начинаем обучение...


training:   2%|▏         | 19/994 [00:26<22:20,  1.37s/it]


KeyboardInterrupt: 

In [ ]:
# 6. Валидация
print("\nНачинаем валидацию...")
pipeline.run_valid()

In [ ]:
# 7. Тестирование и инференс (пример)
print("\nНачинаем тестирование...")
test_split = dataset.get_split("test")
if len(test_split) > 0:
    sample = test_split.get_data(0)
    data = {
        "point": sample["point"].astype(np.float32),
        "full_point": sample["point"].astype(np.float32)
    }
    print("Запуск инференса...")
    result = pipeline.run_inference(data)
    print(f"Размер предсказаний: {result['predict_labels'].shape}")
    print(f"Уникальные классы в предсказаниях: {np.unique(result['predict_labels'])}")